In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW16OT-MakeConcordance-Nehemiah.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""


import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0





Nehemiah01 = [
['Nehemiah','Neh','01','001','1',"The words of Nehemiah the son of Hachaliah. And it came to pass in the month Chisleu, in the twentieth year, as I was in Shushan the palace,"],
['Nehemiah','Neh','01','002','2',"That Hanani, one of my brethren, came, he and \emph{certain} men of Judah; and I asked them concerning the Jews that had escaped, which were left of the captivity, and concerning Jerusalem."],
['Nehemiah','Neh','01','003','3',"And they said unto me, The remnant that are left of the captivity there in the province \emph{are} in great affliction and reproach: the wall of Jerusalem also \emph{is} broken down, and the gates thereof are burned with fire."],
['Nehemiah','Neh','01','004','4',"And it came to pass, when I heard these words, that I sat down and wept, and mourned \emph{certain} days, and fasted, and prayed before the God of heaven,"],
['Nehemiah','Neh','01','005','5',"And said, I beseech thee, O LORD God of heaven, the great and terrible God, that keepeth covenant and mercy for them that love him and observe his commandments:"],
['Nehemiah','Neh','01','006','6',"Let thine ear now be attentive, and thine eyes open, that thou mayest hear the prayer of thy servant, which I pray before thee now, day and night, for the children of Israel thy servants, and confess the sins of the children of Israel, which we have sinned against thee: both I and my father's house have sinned."],
['Nehemiah','Neh','01','007','7',"We have dealt very corruptly against thee, and have not kept the commandments, nor the statutes, nor the judgments, which thou commandedst thy servant Moses."],
['Nehemiah','Neh','01','008','8',"Remember, I beseech thee, the word that thou commandedst thy servant Moses, saying, \emph{If} ye transgress, I will scatter you abroad among the nations:"],
['Nehemiah','Neh','01','009','9',"But \emph{if} ye turn unto me, and keep my commandments, and do them; though there were of you cast out unto the uttermost part of the heaven, \emph{yet} will I gather them from thence, and will bring them unto the place that I have chosen to set my name there."],
['Nehemiah','Neh','01','010','10',"Now these \emph{are} thy servants and thy people, whom thou hast redeemed by thy great power, and by thy strong hand."],
['Nehemiah','Neh','01','011','11',"O Lord, I beseech thee, let now thine ear be attentive to the prayer of thy servant, and to the prayer of thy servants, who desire to fear thy name: and prosper, I pray thee, thy servant this day, and grant him mercy in the sight of this man. For I was the king's cupbearer."]]

Nehemiah02 = [
['Nehemiah','Neh','02','001','1',"And it came to pass in the month Nisan, in the twentieth year of Artaxerxes the king, \emph{that} wine \emph{was} before him: and I took up the wine, and gave \emph{it} unto the king. Now I had not been \emph{beforetime} sad in his presence."],
['Nehemiah','Neh','02','002','2',"Wherefore the king said unto me, Why \emph{is} thy countenance sad, seeing thou \emph{art} not sick? this \emph{is} nothing \emph{else} but sorrow of heart. Then I was very sore afraid,"],
['Nehemiah','Neh','02','003','3',"And said unto the king, Let the king live for ever: why should not my countenance be sad, when the city, the place of my fathers' sepulchres, \emph{lieth} waste, and the gates thereof are consumed with fire?"],
['Nehemiah','Neh','02','004','4',"Then the king said unto me, For what dost thou make request? So I prayed to the God of heaven."],
['Nehemiah','Neh','02','005','5',"And I said unto the king, If it please the king, and if thy servant have found favour in thy sight, that thou wouldest send me unto Judah, unto the city of my fathers' sepulchres, that I may build it."],
['Nehemiah','Neh','02','006','6',"And the king said unto me, (the queen also sitting by him,) For how long shall thy journey be? and when wilt thou return? So it pleased the king to send me; and I set him a time."],
['Nehemiah','Neh','02','007','7',"Moreover I said unto the king, If it please the king, let letters be given me to the governors beyond the river, that they may convey me over till I come into Judah;"],
['Nehemiah','Neh','02','008','8',"And a letter unto Asaph the keeper of the king's forest, that he may give me timber to make beams for the gates of the palace which \emph{appertained} to the house, and for the wall of the city, and for the house that I shall enter into. And the king granted me, according to the good hand of my God upon me."],
['Nehemiah','Neh','02','009','9',"Then I came to the governors beyond the river, and gave them the king's letters. Now the king had sent captains of the army and horsemen with me."],
['Nehemiah','Neh','02','010','10',"When Sanballat the Horonite, and Tobiah the servant, the Ammonite, heard \emph{of} \emph{it}, it grieved them exceedingly that there was come a man to seek the welfare of the children of Israel."],
['Nehemiah','Neh','02','011','11',"So I came to Jerusalem, and was there three days."],
['Nehemiah','Neh','02','012','12',"And I arose in the night, I and some few men with me; neither told I \emph{any} man what my God had put in my heart to do at Jerusalem: neither \emph{was} \emph{there} \emph{any} beast with me, save the beast that I rode upon."],
['Nehemiah','Neh','02','013','13',"And I went out by night by the gate of the valley, even before the dragon well, and to the dung port, and viewed the walls of Jerusalem, which were broken down, and the gates thereof were consumed with fire."],
['Nehemiah','Neh','02','014','14',"Then I went on to the gate of the fountain, and to the king's pool: but \emph{there} \emph{was} no place for the beast \emph{that} \emph{was} under me to pass."],
['Nehemiah','Neh','02','015','15',"Then went I up in the night by the brook, and viewed the wall, and turned back, and entered by the gate of the valley, and \emph{so} returned."],
['Nehemiah','Neh','02','016','16',"And the rulers knew not whither I went, or what I did; neither had I as yet told \emph{it} to the Jews, nor to the priests, nor to the nobles, nor to the rulers, nor to the rest that did the work."],
['Nehemiah','Neh','02','017','17',"Then said I unto them, Ye see the distress that we \emph{are} in, how Jerusalem \emph{lieth} waste, and the gates thereof are burned with fire: come, and let us build up the wall of Jerusalem, that we be no more a reproach."],
['Nehemiah','Neh','02','018','18',"Then I told them of the hand of my God which was good upon me; as also the king's words that he had spoken unto me. And they said, Let us rise up and build. So they strengthened their hands for \emph{this} good \emph{work}."],
['Nehemiah','Neh','02','019','19',"But when Sanballat the Horonite, and Tobiah the servant, the Ammonite, and Geshem the Arabian, heard \emph{it}, they laughed us to scorn, and despised us, and said, What \emph{is} this thing that ye do? will ye rebel against the king?"],
['Nehemiah','Neh','02','020','20',"Then answered I them, and said unto them, The God of heaven, he will prosper us; therefore we his servants will arise and build: but ye have no portion, nor right, nor memorial, in Jerusalem."]
]

Nehemiah03 = [
['Nehemiah','Neh','03','001','1',"Then Eliashib the high priest rose up with his brethren the priests, and they builded the sheep gate; they sanctified it, and set up the doors of it; even unto the tower of Meah they sanctified it, unto the tower of Hananeel."],
['Nehemiah','Neh','03','002','2',"And next unto him builded the men of Jericho. And next to them builded Zaccur the son of Imri."],
['Nehemiah','Neh','03','003','3',"But the fish gate did the sons of Hassenaah build, who \emph{also} laid the beams thereof, and set up the doors thereof, the locks thereof, and the bars thereof."],
['Nehemiah','Neh','03','004','4',"And next unto them repaired Meremoth the son of Urijah, the son of Koz. And next unto them repaired Meshullam the son of Berechiah, the son of Meshezabeel. And next unto them repaired Zadok the son of Baana."],
['Nehemiah','Neh','03','005','5',"And next unto them the Tekoites repaired; but their nobles put not their necks to the work of their Lord."],
['Nehemiah','Neh','03','006','6',"Moreover the old gate repaired Jehoiada the son of Paseah, and Meshullam the son of Besodeiah; they laid the beams thereof, and set up the doors thereof, and the locks thereof, and the bars thereof."],
['Nehemiah','Neh','03','007','7',"And next unto them repaired Melatiah the Gibeonite, and Jadon the Meronothite, the men of Gibeon, and of Mizpah, unto the throne of the governor on this side the river."],
['Nehemiah','Neh','03','008','8',"Next unto him repaired Uzziel the son of Harhaiah, of the goldsmiths. Next unto him also repaired Hananiah the son of \emph{one} \emph{of} the apothecaries, and they fortified Jerusalem unto the broad wall."],
['Nehemiah','Neh','03','009','9',"And next unto them repaired Rephaiah the son of Hur, the ruler of the half part of Jerusalem."],
['Nehemiah','Neh','03','010','10',"And next unto them repaired Jedaiah the son of Harumaph, even over against his house. And next unto him repaired Hattush the son of Hashabniah."],
['Nehemiah','Neh','03','011','11',"Malchijah the son of Harim, and Hashub the son of Pahath-moab, repaired the other piece, and the tower of the furnaces."],
['Nehemiah','Neh','03','012','12',"And next unto him repaired Shallum the son of Halohesh, the ruler of the half part of Jerusalem, he and his daughters."],
['Nehemiah','Neh','03','013','13',"The valley gate repaired Hanun, and the inhabitants of Zanoah; they built it, and set up the doors thereof, the locks thereof, and the bars thereof, and a thousand cubits on the wall unto the dung gate."],
['Nehemiah','Neh','03','014','14',"But the dung gate repaired Malchiah the son of Rechab, the ruler of part of Beth-haccerem; he built it, and set up the doors thereof, the locks thereof, and the bars thereof."],
['Nehemiah','Neh','03','015','15',"But the gate of the fountain repaired Shallun the son of Col-hozeh, the ruler of part of Mizpah; he built it, and covered it, and set up the doors thereof, the locks thereof, and the bars thereof, and the wall of the pool of Siloah by the king's garden, and unto the stairs that go down from the city of David."],
['Nehemiah','Neh','03','016','16',"After him repaired Nehemiah the son of Azbuk, the ruler of the half part of Beth-zur, unto \emph{the} \emph{place} over against the sepulchres of David, and to the pool that was made, and unto the house of the mighty."],
['Nehemiah','Neh','03','017','17',"After him repaired the Levites, Rehum the son of Bani. Next unto him repaired Hashabiah, the ruler of the half part of Keilah, in his part."],
['Nehemiah','Neh','03','018','18',"After him repaired their brethren, Bavai the son of Henadad, the ruler of the half part of Keilah."],
['Nehemiah','Neh','03','019','19',"And next to him repaired Ezer the son of Jeshua, the ruler of Mizpah, another piece over against the going up to the armoury at the turning \emph{of} \emph{the} \emph{wall}."],
['Nehemiah','Neh','03','020','20',"After him Baruch the son of Zabbai earnestly repaired the other piece, from the turning \emph{of} \emph{the} \emph{wall} unto the door of the house of Eliashib the high priest."],
['Nehemiah','Neh','03','021','21',"After him repaired Meremoth the son of Urijah the son of Koz another piece, from the door of the house of Eliashib even to the end of the house of Eliashib."],
['Nehemiah','Neh','03','022','22',"And after him repaired the priests, the men of the plain."],
['Nehemiah','Neh','03','023','23',"After him repaired Benjamin and Hashub over against their house. After him repaired Azariah the son of Maaseiah the son of Ananiah by his house."],
['Nehemiah','Neh','03','024','24',"After him repaired Binnui the son of Henadad another piece, from the house of Azariah unto the turning \emph{of} \emph{the} \emph{wall}, even unto the corner."],
['Nehemiah','Neh','03','025','25',"Palal the son of Uzai, over against the turning \emph{of} \emph{the} \emph{wall}, and the tower which lieth out from the king's high house, that \emph{was} by the court of the prison. After him Pedaiah the son of Parosh."],
['Nehemiah','Neh','03','026','26',"Moreover the Nethinims dwelt in Ophel, unto \emph{the} \emph{place} over against the water gate toward the east, and the tower that lieth out."],
['Nehemiah','Neh','03','027','27',"After them the Tekoites repaired another piece, over against the great tower that lieth out, even unto the wall of Ophel."],
['Nehemiah','Neh','03','028','28',"From above the horse gate repaired the priests, every one over against his house."],
['Nehemiah','Neh','03','029','29',"After them repaired Zadok the son of Immer over against his house. After him repaired also Shemaiah the son of Shechaniah, the keeper of the east gate."],
['Nehemiah','Neh','03','030','30',"After him repaired Hananiah the son of Shelemiah, and Hanun the sixth son of Zalaph, another piece. After him repaired Meshullam the son of Berechiah over against his chamber."],
['Nehemiah','Neh','03','031','31',"After him repaired Malchiah the goldsmith's son unto the place of the Nethinims, and of the merchants, over against the gate Miphkad, and to the going up of the corner."],
['Nehemiah','Neh','03','032','32',"And between the going up of the corner unto the sheep gate repaired the goldsmiths and the merchants."]
]

Nehemiah04 = [
['Nehemiah','Neh','04','001','1',"But it came to pass, that when Sanballat heard that we builded the wall, he was wroth, and took great indignation, and mocked the Jews."],
['Nehemiah','Neh','04','002','2',"And he spake before his brethren and the army of Samaria, and said, What do these feeble Jews? will they fortify themselves? will they sacrifice? will they make an end in a day? will they revive the stones out of the heaps of the rubbish which are burned?"],
['Nehemiah','Neh','04','003','3',"Now Tobiah the Ammonite \emph{was} by him, and he said, Even that which they build, if a fox go up, he shall even break down their stone wall."],
['Nehemiah','Neh','04','004','4',"Hear, O our God; for we are despised: and turn their reproach upon their own head, and give them for a prey in the land of captivity:"],
['Nehemiah','Neh','04','005','5',"And cover not their iniquity, and let not their sin be blotted out from before thee: for they have provoked \emph{thee} to anger before the builders."],
['Nehemiah','Neh','04','006','6',"So built we the wall; and all the wall was joined together unto the half thereof: for the people had a mind to work."],
['Nehemiah','Neh','04','007','7',"But it came to pass, \emph{that} when Sanballat, and Tobiah, and the Arabians, and the Ammonites, and the Ashdodites, heard that the walls of Jerusalem were made up, \emph{and} that the breaches began to be stopped, then they were very wroth,"],
['Nehemiah','Neh','04','008','8',"And conspired all of them together to come \emph{and} to fight against Jerusalem, and to hinder it."],
['Nehemiah','Neh','04','009','9',"Nevertheless we made our prayer unto our God, and set a watch against them day and night, because of them."],
['Nehemiah','Neh','04','010','10',"And Judah said, The strength of the bearers of burdens is decayed, and \emph{there} \emph{is} much rubbish; so that we are not able to build the wall."],
['Nehemiah','Neh','04','011','11',"And our adversaries said, They shall not know, neither see, till we come in the midst among them, and slay them, and cause the work to cease."],
['Nehemiah','Neh','04','012','12',"And it came to pass, that when the Jews which dwelt by them came, they said unto us ten times, From all places whence ye shall return unto us \emph{they} \emph{will} \emph{be} \emph{upon} \emph{you}."],
['Nehemiah','Neh','04','013','13',"Therefore set I in the lower places behind the wall, \emph{and} on the higher places, I even set the people after their families with their swords, their spears, and their bows."],
['Nehemiah','Neh','04','014','14',"And I looked, and rose up, and said unto the nobles, and to the rulers, and to the rest of the people, Be not ye afraid of them: remember the Lord, \emph{which} \emph{is} great and terrible, and fight for your brethren, your sons, and your daughters, your wives, and your houses."],
['Nehemiah','Neh','04','015','15',"And it came to pass, when our enemies heard that it was known unto us, and God had brought their counsel to nought, that we returned all of us to the wall, every one unto his work."],
['Nehemiah','Neh','04','016','16',"And it came to pass from that time forth, \emph{that} the half of my servants wrought in the work, and the other half of them held both the spears, the shields, and the bows, and the habergeons; and the rulers \emph{were} behind all the house of Judah."],
['Nehemiah','Neh','04','017','17',"They which builded on the wall, and they that bare burdens, with those that laded, \emph{every} \emph{one} with one of his hands wrought in the work, and with the other \emph{hand} held a weapon."],
['Nehemiah','Neh','04','018','18',"For the builders, every one had his sword girded by his side, and \emph{so} builded. And he that sounded the trumpet \emph{was} by me."],
['Nehemiah','Neh','04','019','19',"And I said unto the nobles, and to the rulers, and to the rest of the people, The work \emph{is} great and large, and we are separated upon the wall, one far from another."],
['Nehemiah','Neh','04','020','20',"In what place \emph{therefore} ye hear the sound of the trumpet, resort ye thither unto us: our God shall fight for us."],
['Nehemiah','Neh','04','021','21',"So we laboured in the work: and half of them held the spears from the rising of the morning till the stars appeared."],
['Nehemiah','Neh','04','022','22',"Likewise at the same time said I unto the people, Let every one with his servant lodge within Jerusalem, that in the night they may be a guard to us, and labour on the day."],
['Nehemiah','Neh','04','023','23',"So neither I, nor my brethren, nor my servants, nor the men of the guard which followed me, none of us put off our clothes, \emph{saving} \emph{that} every one put them off for washing."]
]

Nehemiah05 = [
['Nehemiah','Neh','05','001','1',"And there was a great cry of the people and of their wives against their brethren the Jews."],
['Nehemiah','Neh','05','002','2',"For there were that said, We, our sons, and our daughters, \emph{are} many: therefore we take up corn \emph{for} \emph{them}, that we may eat, and live."],
['Nehemiah','Neh','05','003','3',"\emph{Some} also there were that said, We have mortgaged our lands, vineyards, and houses, that we might buy corn, because of the dearth."],
['Nehemiah','Neh','05','004','4',"There were also that said, We have borrowed money for the king's tribute, \emph{and} \emph{that} \emph{upon} our lands and vineyards."],
['Nehemiah','Neh','05','005','5',"Yet now our flesh \emph{is} as the flesh of our brethren, our children as their children: and, lo, we bring into bondage our sons and our daughters to be servants, and \emph{some} of our daughters are brought unto bondage \emph{already}: neither \emph{is} \emph{it} in our power \emph{to} \emph{redeem} \emph{them}; for other men have our lands and vineyards."],
['Nehemiah','Neh','05','006','6',"And I was very angry when I heard their cry and these words."],
['Nehemiah','Neh','05','007','7',"Then I consulted with myself, and I rebuked the nobles, and the rulers, and said unto them, Ye exact usury, every one of his brother. And I set a great assembly against them."],
['Nehemiah','Neh','05','008','8',"And I said unto them, We after our ability have redeemed our brethren the Jews, which were sold unto the heathen; and will ye even sell your brethren? or shall they be sold unto us? Then held they their peace, and found nothing \emph{to} \emph{answer}."],
['Nehemiah','Neh','05','009','9',"Also I said, It \emph{is} not good that ye do: ought ye not to walk in the fear of our God because of the reproach of the heathen our enemies?"],
['Nehemiah','Neh','05','010','10',"I likewise, \emph{and} my brethren, and my servants, might exact of them money and corn: I pray you, let us leave off this usury."],
['Nehemiah','Neh','05','011','11',"Restore, I pray you, to them, even this day, their lands, their vineyards, their oliveyards, and their houses, also the hundredth \emph{part} of the money, and of the corn, the wine, and the oil, that ye exact of them."],
['Nehemiah','Neh','05','012','12',"Then said they, We will restore \emph{them}, and will require nothing of them; so will we do as thou sayest. Then I called the priests, and took an oath of them, that they should do according to this promise."],
['Nehemiah','Neh','05','013','13',"Also I shook my lap, and said, So God shake out every man from his house, and from his labour, that performeth not this promise, even thus be he shaken out, and emptied. And all the congregation said, Amen, and praised the LORD. And the people did according to this promise."],
['Nehemiah','Neh','05','014','14',"Moreover from the time that I was appointed to be their governor in the land of Judah, from the twentieth year even unto the two and thirtieth year of Artaxerxes the king, \emph{that} \emph{is}, twelve years, I and my brethren have not eaten the bread of the governor."],
['Nehemiah','Neh','05','015','15',"But the former governors that \emph{had} \emph{been} before me were chargeable unto the people, and had taken of them bread and wine, beside forty shekels of silver; yea, even their servants bare rule over the people: but so did not I, because of the fear of God."],
['Nehemiah','Neh','05','016','16',"Yea, also I continued in the work of this wall, neither bought we any land: and all my servants \emph{were} gathered thither unto the work."],
['Nehemiah','Neh','05','017','17',"Moreover \emph{there} \emph{were} at my table an hundred and fifty of the Jews and rulers, beside those that came unto us from among the heathen that \emph{are} about us."],
['Nehemiah','Neh','05','018','18',"Now \emph{that} which was prepared \emph{for} \emph{me} daily \emph{was} one ox \emph{and} six choice sheep; also fowls were prepared for me, and once in ten days store of all sorts of wine: yet for all this required not I the bread of the governor, because the bondage was heavy upon this people."],
['Nehemiah','Neh','05','019','19',"Think upon me, my God, for good, \emph{according} to all that I have done for this people."]
]

Nehemiah06 = [
['Nehemiah','Neh','06','001','1',"Now it came to pass, when Sanballat, and Tobiah, and Geshem the Arabian, and the rest of our enemies, heard that I had builded the wall, and \emph{that} there was no breach left therein; (though at that time I had not set up the doors upon the gates;)"],
['Nehemiah','Neh','06','002','2',"That Sanballat and Geshem sent unto me, saying, Come, let us meet together in \emph{some} \emph{one} \emph{of} the villages in the plain of Ono. But they thought to do me mischief."],
['Nehemiah','Neh','06','003','3',"And I sent messengers unto them, saying, I \emph{am} doing a great work, so that I cannot come down: why should the work cease, whilst I leave it, and come down to you?"],
['Nehemiah','Neh','06','004','4',"Yet they sent unto me four times after this sort; and I answered them after the same manner."],
['Nehemiah','Neh','06','005','5',"Then sent Sanballat his servant unto me in like manner the fifth time with an open letter in his hand;"],
['Nehemiah','Neh','06','006','6',"Wherein \emph{was} written, It is reported among the heathen, and Gashmu saith \emph{it,} \emph{that} thou and the Jews think to rebel: for which cause thou buildest the wall, that thou mayest be their king, according to these words."],
['Nehemiah','Neh','06','007','7',"And thou hast also appointed prophets to preach of thee at Jerusalem, saying, \emph{There} \emph{is} a king in Judah: and now shall it be reported to the king according to these words. Come now therefore, and let us take counsel together."],
['Nehemiah','Neh','06','008','8',"Then I sent unto him, saying, There are no such things done as thou sayest, but thou feignest them out of thine own heart."],
['Nehemiah','Neh','06','009','9',"For they all made us afraid, saying, Their hands shall be weakened from the work, that it be not done. Now therefore, \emph{O} \emph{God}, strengthen my hands."],
['Nehemiah','Neh','06','010','10',"Afterward I came unto the house of Shemaiah the son of Delaiah the son of Mehetabeel, who \emph{was} shut up; and he said, Let us meet together in the house of God, within the temple, and let us shut the doors of the temple: for they will come to slay thee; yea, in the night will they come to slay thee."],
['Nehemiah','Neh','06','011','11',"And I said, Should such a man as I flee? and who \emph{is} \emph{there}, that, \emph{being} as I \emph{am}, would go into the temple to save his life? I will not go in."],
['Nehemiah','Neh','06','012','12',"And, lo, I perceived that God had not sent him; but that he pronounced this prophecy against me: for Tobiah and Sanballat had hired him."],
['Nehemiah','Neh','06','013','13',"Therefore \emph{was} he hired, that I should be afraid, and do so, and sin, and \emph{that} they might have \emph{matter} for an evil report, that they might reproach me."],
['Nehemiah','Neh','06','014','14',"My God, think thou upon Tobiah and Sanballat according to these their works, and on the prophetess Noadiah, and the rest of the prophets, that would have put me in fear."],
['Nehemiah','Neh','06','015','15',"So the wall was finished in the twenty and fifth \emph{day} of \emph{the} \emph{month} Elul, in fifty and two days."],
['Nehemiah','Neh','06','016','16',"And it came to pass, that when all our enemies heard \emph{thereof}, and all the heathen that \emph{were} about us saw \emph{these} \emph{things}, they were much cast down in their own eyes: for they perceived that this work was wrought of our God."],
['Nehemiah','Neh','06','017','17',"Moreover in those days the nobles of Judah sent many letters unto Tobiah, and \emph{the} \emph{letters} of Tobiah came unto them."],
['Nehemiah','Neh','06','018','18',"For \emph{there} \emph{were} many in Judah sworn unto him, because he \emph{was} the son in law of Shechaniah the son of Arah; and his son Johanan had taken the daughter of Meshullam the son of Berechiah."],
['Nehemiah','Neh','06','019','19',"Also they reported his good deeds before me, and uttered my words to him. \emph{And} Tobiah sent letters to put me in fear."]
]

Nehemiah07 = [
    ['Nehemiah','Neh','07','001','1',"Now it came to pass, when the wall was built, and I had set up the doors, and the porters and the singers and the Levites were appointed,"],
    ['Nehemiah','Neh','07','002','2',"That I gave my brother Hanani, and Hananiah the ruler of the palace, charge over Jerusalem: for he \emph{was} a faithful man, and feared God above many."],
    ['Nehemiah','Neh','07','003','3',"And I said unto them, Let not the gates of Jerusalem be opened until the sun be hot; and while they stand by, let them shut the doors, and bar \emph{them}: and appoint watches of the inhabitants of Jerusalem, every one in his watch, and every one \emph{to} \emph{be} over against his house."],
    ['Nehemiah','Neh','07','004','4',"Now the city \emph{was} large and great: but the people \emph{were} few therein, and the houses \emph{were} not builded."],
    ['Nehemiah','Neh','07','005','5',"And my God put into mine heart to gather together the nobles, and the rulers, and the people, that they might be reckoned by genealogy. And I found a register of the genealogy of them which came up at the first, and found written therein,"],
    ['Nehemiah','Neh','07','006','6',"These \emph{are} the children of the province, that went up out of the captivity, of those that had been carried away, whom Nebuchadnezzar the king of Babylon had carried away, and came again to Jerusalem and to Judah, every one unto his city;"],
    ['Nehemiah','Neh','07','007','7',"Who came with Zerubbabel, Jeshua, Nehemiah, Azariah, Raamiah, Nahamani, Mordecai, Bilshan, Mispereth, Bigvai, Nehum, Baanah. The number, \emph{I} \emph{say}, of the men of the people of Israel \emph{was} \emph{this};"],
    ['Nehemiah','Neh','07','008','8',"The children of Parosh, two thousand an hundred seventy and two."],
    ['Nehemiah','Neh','07','009','9',"The children of Shephatiah, three hundred seventy and two."],
    ['Nehemiah','Neh','07','010','10',"The children of Arah, six hundred fifty and two."],
['Nehemiah','Neh','07','011','11',"The children of Pahath-moab, of the children of Jeshua and Joab, two thousand and eight hundred \emph{and} eighteen."],
['Nehemiah','Neh','07','012','12',"The children of Elam, a thousand two hundred fifty and four."],
['Nehemiah','Neh','07','013','13',"The children of Zattu, eight hundred forty and five."],
['Nehemiah','Neh','07','014','14',"The children of Zaccai, seven hundred and threescore."],
['Nehemiah','Neh','07','015','15',"The children of Binnui, six hundred forty and eight."],
['Nehemiah','Neh','07','016','16',"The children of Bebai, six hundred twenty and eight."],
['Nehemiah','Neh','07','017','17',"The children of Azgad, two thousand three hundred twenty and two."],
['Nehemiah','Neh','07','018','18',"The children of Adonikam, six hundred threescore and seven."],
['Nehemiah','Neh','07','019','19',"The children of Bigvai, two thousand threescore and seven."],
['Nehemiah','Neh','07','020','20',"The children of Adin, six hundred fifty and five."],
['Nehemiah','Neh','07','021','21',"The children of Ater of Hezekiah, ninety and eight."],
['Nehemiah','Neh','07','022','22',"The children of Hashum, three hundred twenty and eight."],
['Nehemiah','Neh','07','023','23',"The children of Bezai, three hundred twenty and four."],
['Nehemiah','Neh','07','024','24',"The children of Hariph, an hundred and twelve."],
['Nehemiah','Neh','07','025','25',"The children of Gibeon, ninety and five."],
['Nehemiah','Neh','07','026','26',"The men of Beth-lehem and Netophah, an hundred fourscore and eight."],
['Nehemiah','Neh','07','027','27',"The men of Anathoth, an hundred twenty and eight."],
['Nehemiah','Neh','07','028','28',"The men of Beth-azmaveth, forty and two."],
['Nehemiah','Neh','07','029','29',"The men of Kirjath-jearim, Chephirah, and Beeroth, seven hundred forty and three."],
['Nehemiah','Neh','07','030','30',"The men of Ramah and Geba, six hundred twenty and one."],
['Nehemiah','Neh','07','031','31',"The men of Michmas, an hundred and twenty and two."],
['Nehemiah','Neh','07','032','32',"The men of Beth-el and Ai, an hundred twenty and three."],
['Nehemiah','Neh','07','033','33',"The men of the other Nebo, fifty and two."],
['Nehemiah','Neh','07','034','34',"The children of the other Elam, a thousand two hundred fifty and four."],
['Nehemiah','Neh','07','035','35',"The children of Harim, three hundred and twenty."],
['Nehemiah','Neh','07','036','36',"The children of Jericho, three hundred forty and five."],
['Nehemiah','Neh','07','037','37',"The children of Lod, Hadid, and Ono, seven hundred twenty and one."],
['Nehemiah','Neh','07','038','38',"The children of Senaah, three thousand nine hundred and thirty."],
['Nehemiah','Neh','07','039','39',"The priests: the children of Jedaiah, of the house of Jeshua, nine hundred seventy and three."],
['Nehemiah','Neh','07','040','40',"The children of Immer, a thousand fifty and two."],
['Nehemiah','Neh','07','041','41',"The children of Pashur, a thousand two hundred forty and seven."],
['Nehemiah','Neh','07','042','42',"The children of Harim, a thousand and seventeen."],
['Nehemiah','Neh','07','043','43',"The Levites: the children of Jeshua, of Kadmiel, \emph{and} of the children of Hodevah, seventy and four."],
['Nehemiah','Neh','07','044','44',"The singers: the children of Asaph, an hundred forty and eight."],
['Nehemiah','Neh','07','045','45',"The porters: the children of Shallum, the children of Ater, the children of Talmon, the children of Akkub, the children of Hatita, the children of Shobai, an hundred thirty and eight."],
['Nehemiah','Neh','07','046','46',"The Nethinims: the children of Ziha, the children of Hashupha, the children of Tabbaoth,"],
['Nehemiah','Neh','07','047','47',"The children of Keros, the children of Sia, the children of Padon,"],
['Nehemiah','Neh','07','048','48',"The children of Lebana, the children of Hagaba, the children of Shalmai,"],
['Nehemiah','Neh','07','049','49',"The children of Hanan, the children of Giddel, the children of Gahar,"],
['Nehemiah','Neh','07','050','50',"The children of Reaiah, the children of Rezin, the children of Nekoda,"],
['Nehemiah','Neh','07','051','51',"The children of Gazzam, the children of Uzza, the children of Phaseah,"],
['Nehemiah','Neh','07','052','52',"The children of Besai, the children of Meunim, the children of Nephishesim,"],
['Nehemiah','Neh','07','053','53',"The children of Bakbuk, the children of Hakupha, the children of Harhur,"],
['Nehemiah','Neh','07','054','54',"The children of Bazlith, the children of Mehida, the children of Harsha,"],
['Nehemiah','Neh','07','055','55',"The children of Barkos, the children of Sisera, the children of Tamah,"],
['Nehemiah','Neh','07','056','56',"The children of Neziah, the children of Hatipha."],
['Nehemiah','Neh','07','057','57',"The children of Solomon's servants: the children of Sotai, the children of Sophereth, the children of Perida,"],
['Nehemiah','Neh','07','058','58',"The children of Jaala, the children of Darkon, the children of Giddel,"],
['Nehemiah','Neh','07','059','59',"The children of Shephatiah, the children of Hattil, the children of Pochereth of Zebaim, the children of Amon."],
    ['Nehemiah','Neh','07','060','60',"All the Nethinims, and the children of Solomon's servants, \emph{were} three hundred ninety and two."],
    ['Nehemiah','Neh','07','061','61',"And these \emph{were} they which went up \emph{also} from Tel-melah, Tel-haresha, Cherub, Addon, and Immer: but they could not shew their father's house, nor their seed, whether they \emph{were} of Israel."],
    ['Nehemiah','Neh','07','062','62',"The children of Delaiah, the children of Tobiah, the children of Nekoda, six hundred forty and two."],
    ['Nehemiah','Neh','07','063','63',"And of the priests: the children of Habaiah, the children of Koz, the children of Barzillai, which took \emph{one} of the daughters of Barzillai the Gileadite to wife, and was called after their name."],
    ['Nehemiah','Neh','07','064','64',"These sought their register \emph{among} those that were reckoned by genealogy, but it was not found: therefore were they, as polluted, put from the priesthood."],
    ['Nehemiah','Neh','07','065','65',"And the Tirshatha said unto them, that they should not eat of the most holy things, till there stood \emph{up} a priest with Urim and Thummim."],
    ['Nehemiah','Neh','07','066','66',"The whole congregation together \emph{was} forty and two thousand three hundred and threescore,"],
    ['Nehemiah','Neh','07','067','67',"Beside their manservants and their maidservants, of whom \emph{there} \emph{were} seven thousand three hundred thirty and seven: and they had two hundred forty and five singing men and singing women."],
    ['Nehemiah','Neh','07','068','68',"Their horses, seven hundred thirty and six: their mules, two hundred forty and five:"],
    ['Nehemiah','Neh','07','069','69',"\emph{Their} camels, four hundred thirty and five: six thousand seven hundred and twenty asses."],
    ['Nehemiah','Neh','07','070','70',"And some of the chief of the fathers gave unto the work. The Tirshatha gave to the treasure a thousand drams of gold, fifty basons, five hundred and thirty priests' garments."],
    ['Nehemiah','Neh','07','071','71',"And \emph{some} of the chief of the fathers gave to the treasure of the work twenty thousand drams of gold, and two thousand and two hundred pound of silver."],
    ['Nehemiah','Neh','07','072','72',"And \emph{that} which the rest of the people gave \emph{was} twenty thousand drams of gold, and two thousand pound of silver, and threescore and seven priests' garments."],
    ['Nehemiah','Neh','07','073','73',"So the priests, and the Levites, and the porters, and the singers, and \emph{some} of the people, and the Nethinims, and all Israel, dwelt in their cities; and when the seventh month came, the children of Israel \emph{were} in their cities."]
]

Nehemiah08 = [
   ['Nehemiah','Neh','08','001','1',"And all the people gathered themselves together as one man into the street that \emph{was} before the water gate; and they spake unto Ezra the scribe to bring the book of the law of Moses, which the LORD had commanded to Israel."],
   ['Nehemiah','Neh','08','002','2',"And Ezra the priest brought the law before the congregation both of men and women, and all that could hear with understanding, upon the first day of the seventh month."],
   ['Nehemiah','Neh','08','003','3',"And he read therein before the street that \emph{was} before the water gate from the morning until midday, before the men and the women, and those that could understand; and the ears of all the people \emph{were} \emph{attentive} unto the book of the law."],
   ['Nehemiah','Neh','08','004','4',"And Ezra the scribe stood upon a pulpit of wood, which they had made for the purpose; and beside him stood Mattithiah, and Shema, and Anaiah, and Urijah, and Hilkiah, and Maaseiah, on his right hand; and on his left hand, Pedaiah, and Mishael, and Malchiah, and Hashum, and Hashbadana, Zechariah, \emph{and} Meshullam."],
   ['Nehemiah','Neh','08','005','5',"And Ezra opened the book in the sight of all the people; (for he was above all the people;) and when he opened it, all the people stood up:"],
   ['Nehemiah','Neh','08','006','6',"And Ezra blessed the LORD, the great God. And all the people answered, Amen, Amen, with lifting up their hands: and they bowed their heads, and worshipped the LORD with \emph{their} faces to the ground."],
   ['Nehemiah','Neh','08','007','7',"Also Jeshua, and Bani, and Sherebiah, Jamin, Akkub, Shabbethai, Hodijah, Maaseiah, Kelita, Azariah, Jozabad, Hanan, Pelaiah, and the Levites, caused the people to understand the law: and the people \emph{stood} in their place."],
   ['Nehemiah','Neh','08','008','8',"So they read in the book in the law of God distinctly, and gave the sense, and caused \emph{them} to understand the reading."],
    ['Nehemiah','Neh','08','009','9',"And Nehemiah, which \emph{is} the Tirshatha, and Ezra the priest the scribe, and the Levites that taught the people, said unto all the people, This day \emph{is} holy unto the LORD your God; mourn not, nor weep. For all the people wept, when they heard the words of the law."],
    ['Nehemiah','Neh','08','010','10',"Then he said unto them, Go your way, eat the fat, and drink the sweet, and send portions unto them for whom nothing is prepared: for \emph{this} day \emph{is} holy unto our Lord: neither be ye sorry; for the joy of the LORD is your strength."],
    ['Nehemiah','Neh','08','011','11',"So the Levites stilled all the people, saying, Hold your peace, for the day \emph{is} holy; neither be ye grieved."],
    ['Nehemiah','Neh','08','012','12',"And all the people went their way to eat, and to drink, and to send portions, and to make great mirth, because they had understood the words that were declared unto them."],
    ['Nehemiah','Neh','08','013','13',"And on the second day were gathered together the chief of the fathers of all the people, the priests, and the Levites, unto Ezra the scribe, even to understand the words of the law."],
    ['Nehemiah','Neh','08','014','14',"And they found written in the law which the LORD had commanded by Moses, that the children of Israel should dwell in booths in the feast of the seventh month:"],
    ['Nehemiah','Neh','08','015','15',"And that they should publish and proclaim in all their cities, and in Jerusalem, saying, Go forth unto the mount, and fetch olive branches, and pine branches, and myrtle branches, and palm branches, and branches of thick trees, to make booths, as \emph{it} \emph{is} written."],
    ['Nehemiah','Neh','08','016','16',"So the people went forth, and brought \emph{them}, and made themselves booths, every one upon the roof of his house, and in their courts, and in the courts of the house of God, and in the street of the water gate, and in the street of the gate of Ephraim."],
    ['Nehemiah','Neh','08','017','17',"And all the congregation of them that were come again out of the captivity made booths, and sat under the booths: for since the days of Jeshua the son of Nun unto that day had not the children of Israel done so. And there was very great gladness."],
    ['Nehemiah','Neh','08','018','18',"Also day by day, from the first day unto the last day, he read in the book of the law of God. And they kept the feast seven days; and on the eighth day \emph{was} a solemn assembly, according unto the manner."]
]

Nehemiah09 = [
['Nehemiah','Neh','09','001','1',"Now in the twenty and fourth day of this month the children of Israel were assembled with fasting, and with sackclothes, and earth upon them."],
['Nehemiah','Neh','09','002','2',"And the seed of Israel separated themselves from all strangers, and stood and confessed their sins, and the iniquities of their fathers."],
['Nehemiah','Neh','09','003','3',"And they stood up in their place, and read in the book of the law of the LORD their God \emph{one} fourth part of the day; and \emph{another} fourth part they confessed, and worshipped the LORD their God."],
['Nehemiah','Neh','09','004','4',"Then stood up upon the stairs, of the Levites, Jeshua, and Bani, Kadmiel, Shebaniah, Bunni, Sherebiah, Bani, \emph{and} Chenani, and cried with a loud voice unto the LORD their God."],
['Nehemiah','Neh','09','005','5',"Then the Levites, Jeshua, and Kadmiel, Bani, Hashabniah, Sherebiah, Hodijah, Shebaniah, \emph{and} Pethahiah, said, Stand up \emph{and} bless the LORD your God for ever and ever: and blessed be thy glorious name, which is exalted above all blessing and praise."],
['Nehemiah','Neh','09','006','6',"Thou, \emph{even} thou, \emph{art} LORD alone; thou hast made heaven, the heaven of heavens, with all their host, the earth, and all \emph{things} that \emph{are} therein, the seas, and all that \emph{is} therein, and thou preservest them all; and the host of heaven worshippeth thee."],
['Nehemiah','Neh','09','007','7',"Thou \emph{art} the LORD the God, who didst choose Abram, and broughtest him forth out of Ur of the Chaldees, and gavest him the name of Abraham;"],
['Nehemiah','Neh','09','008','8',"And foundest his heart faithful before thee, and madest a covenant with him to give the land of the Canaanites, the Hittites, the Amorites, and the Perizzites, and the Jebusites, and the Girgashites, to give \emph{it,} \emph{I} \emph{say}, to his seed, and hast performed thy words; for thou \emph{art} righteous:"],
['Nehemiah','Neh','09','009','9',"And didst see the affliction of our fathers in Egypt, and heardest their cry by the Red sea;"],
['Nehemiah','Neh','09','010','10',"And shewedst signs and wonders upon Pharaoh, and on all his servants, and on all the people of his land: for thou knewest that they dealt proudly against them. So didst thou get thee a name, as \emph{it} \emph{is} this day."],
['Nehemiah','Neh','09','011','11',"And thou didst divide the sea before them, so that they went through the midst of the sea on the dry land; and their persecutors thou threwest into the deeps, as a stone into the mighty waters."],
['Nehemiah','Neh','09','012','12',"Moreover thou leddest them in the day by a cloudy pillar; and in the night by a pillar of fire, to give them light in the way wherein they should go."],
['Nehemiah','Neh','09','013','13',"Thou camest down also upon mount Sinai, and spakest with them from heaven, and gavest them right judgments, and true laws, good statutes and commandments:"],
['Nehemiah','Neh','09','014','14',"And madest known unto them thy holy sabbath, and commandedst them precepts, statutes, and laws, by the hand of Moses thy servant:"],
['Nehemiah','Neh','09','015','15',"And gavest them bread from heaven for their hunger, and broughtest forth water for them out of the rock for their thirst, and promisedst them that they should go in to possess the land which thou hadst sworn to give them."],
['Nehemiah','Neh','09','016','16',"But they and our fathers dealt proudly, and hardened their necks, and hearkened not to thy commandments,"],
['Nehemiah','Neh','09','017','17',"And refused to obey, neither were mindful of thy wonders that thou didst among them; but hardened their necks, and in their rebellion appointed a captain to return to their bondage: but thou \emph{art} a God ready to pardon, gracious and merciful, slow to anger, and of great kindness, and forsookest them not."],
['Nehemiah','Neh','09','018','18',"Yea, when they had made them a molten calf, and said, This \emph{is} thy God that brought thee up out of Egypt, and had wrought great provocations;"],
['Nehemiah','Neh','09','019','19',"Yet thou in thy manifold mercies forsookest them not in the wilderness: the pillar of the cloud departed not from them by day, to lead them in the way; neither the pillar of fire by night, to shew them light, and the way wherein they should go."],
['Nehemiah','Neh','09','020','20',"Thou gavest also thy good spirit to instruct them, and withheldest not thy manna from their mouth, and gavest them water for their thirst."],
['Nehemiah','Neh','09','021','21',"Yea, forty years didst thou sustain them in the wilderness, \emph{so} \emph{that} they lacked nothing; their clothes waxed not old, and their feet swelled not."],
['Nehemiah','Neh','09','022','22',"Moreover thou gavest them kingdoms and nations, and didst divide them into corners: so they possessed the land of Sihon, and the land of the king of Heshbon, and the land of Og king of Bashan."],
['Nehemiah','Neh','09','023','23',"Their children also multipliedst thou as the stars of heaven, and broughtest them into the land, concerning which thou hadst promised to their fathers, that they should go in to possess \emph{it}."],
['Nehemiah','Neh','09','024','24',"So the children went in and possessed the land, and thou subduedst before them the inhabitants of the land, the Canaanites, and gavest them into their hands, with their kings, and the people of the land, that they might do with them as they would."],
['Nehemiah','Neh','09','025','25',"And they took strong cities, and a fat land, and possessed houses full of all goods, wells digged, vineyards, and oliveyards, and fruit trees in abundance: so they did eat, and were filled, and became fat, and delighted themselves in thy great goodness."],
['Nehemiah','Neh','09','026','26',"Nevertheless they were disobedient, and rebelled against thee, and cast thy law behind their backs, and slew thy prophets which testified against them to turn them to thee, and they wrought great provocations."],
['Nehemiah','Neh','09','027','27',"Therefore thou deliveredst them into the hand of their enemies, who vexed them: and in the time of their trouble, when they cried unto thee, thou heardest \emph{them} from heaven; and according to thy manifold mercies thou gavest them saviours, who saved them out of the hand of their enemies."],
['Nehemiah','Neh','09','028','28',"But after they had rest, they did evil again before thee: therefore leftest thou them in the hand of their enemies, so that they had the dominion over them: yet when they returned, and cried unto thee, thou heardest \emph{them} from heaven; and many times didst thou deliver them according to thy mercies;"],
['Nehemiah','Neh','09','029','29',"And testifiedst against them, that thou mightest bring them again unto thy law: yet they dealt proudly, and hearkened not unto thy commandments, but sinned against thy judgments, (which if a man do, he shall live in them;) and withdrew the shoulder, and hardened their neck, and would not hear."],
['Nehemiah','Neh','09','030','30',"Yet many years didst thou forbear them, and testifiedst against them by thy spirit in thy prophets: yet would they not give ear: therefore gavest thou them into the hand of the people of the lands."],
['Nehemiah','Neh','09','031','31',"Nevertheless for thy great mercies' sake thou didst not utterly consume them, nor forsake them; for thou \emph{art} a gracious and merciful God."],
['Nehemiah','Neh','09','032','32',"Now therefore, our God, the great, the mighty, and the terrible God, who keepest covenant and mercy, let not all the trouble seem little before thee, that hath come upon us, on our kings, on our princes, and on our priests, and on our prophets, and on our fathers, and on all thy people, since the time of the kings of Assyria unto this day."],
['Nehemiah','Neh','09','033','33',"Howbeit thou \emph{art} just in all that is brought upon us; for thou hast done right, but we have done wickedly:"],
['Nehemiah','Neh','09','034','34',"Neither have our kings, our princes, our priests, nor our fathers, kept thy law, nor hearkened unto thy commandments and thy testimonies, wherewith thou didst testify against them."],
['Nehemiah','Neh','09','035','35',"For they have not served thee in their kingdom, and in thy great goodness that thou gavest them, and in the large and fat land which thou gavest before them, neither turned they from their wicked works."],
['Nehemiah','Neh','09','036','36',"Behold, we \emph{are} servants this day, and \emph{for} the land that thou gavest unto our fathers to eat the fruit thereof and the good thereof, behold, we \emph{are} servants in it:"],
['Nehemiah','Neh','09','037','37',"And it yieldeth much increase unto the kings whom thou hast set over us because of our sins: also they have dominion over our bodies, and over our cattle, at their pleasure, and we \emph{are} in great distress."],
['Nehemiah','Neh','09','038','38',"And because of all this we make a sure \emph{covenant}, and write \emph{it}; and our princes, Levites, \emph{and} priests, seal \emph{unto} \emph{it}."]
]

Nehemiah10 = [
['Nehemiah','Neh','10','001','1',"Now those that sealed \emph{were}, Nehemiah, the Tirshatha, the son of Hachaliah, and Zidkijah,"],
['Nehemiah','Neh','10','002','2',"Seraiah, Azariah, Jeremiah,"],
['Nehemiah','Neh','10','003','3',"Pashur, Amariah, Malchijah,"],
['Nehemiah','Neh','10','004','4',"Hattush, Shebaniah, Malluch,"],
['Nehemiah','Neh','10','005','5',"Harim, Meremoth, Obadiah,"],
['Nehemiah','Neh','10','006','6',"Daniel, Ginnethon, Baruch,"],
['Nehemiah','Neh','10','007','7',"Meshullam, Abijah, Mijamin,"],
['Nehemiah','Neh','10','008','8',"Maaziah, Bilgai, Shemaiah: these \emph{were} the priests."],
['Nehemiah','Neh','10','009','9',"And the Levites: both Jeshua the son of Azaniah, Binnui of the sons of Henadad, Kadmiel;"],
['Nehemiah','Neh','10','010','10',"And their brethren, Shebaniah, Hodijah, Kelita, Pelaiah, Hanan,"],
['Nehemiah','Neh','10','011','11',"Micha, Rehob, Hashabiah,"],
['Nehemiah','Neh','10','012','12',"Zaccur, Sherebiah, Shebaniah,"],
['Nehemiah','Neh','10','013','13',"Hodijah, Bani, Beninu."],
['Nehemiah','Neh','10','014','14',"The chief of the people; Parosh, Pahath-moab, Elam, Zatthu, Bani,"],
['Nehemiah','Neh','10','015','15',"Bunni, Azgad, Bebai,"],
['Nehemiah','Neh','10','016','16',"Adonijah, Bigvai, Adin,"],
['Nehemiah','Neh','10','017','17',"Ater, Hizkijah, Azzur,"],
['Nehemiah','Neh','10','018','18',"Hodijah, Hashum, Bezai,"],
['Nehemiah','Neh','10','019','19',"Hariph, Anathoth, Nebai,"],
['Nehemiah','Neh','10','020','20',"Magpiash, Meshullam, Hezir,"],
['Nehemiah','Neh','10','021','21',"Meshezabeel, Zadok, Jaddua,"],
['Nehemiah','Neh','10','022','22',"Pelatiah, Hanan, Anaiah,"],
['Nehemiah','Neh','10','023','23',"Hoshea, Hananiah, Hashub,"],
['Nehemiah','Neh','10','024','24',"Hallohesh, Pileha, Shobek,"],
['Nehemiah','Neh','10','025','25',"Rehum, Hashabnah, Maaseiah,"],
['Nehemiah','Neh','10','026','26',"And Ahijah, Hanan, Anan,"],
['Nehemiah','Neh','10','027','27',"Malluch, Harim, Baanah."],
['Nehemiah','Neh','10','028','28',"And the rest of the people, the priests, the Levites, the porters, the singers, the Nethinims, and all they that had separated themselves from the people of the lands unto the law of God, their wives, their sons, and their daughters, every one having knowledge, and having understanding;"],
['Nehemiah','Neh','10','029','29',"They clave to their brethren, their nobles, and entered into a curse, and into an oath, to walk in God's law, which was given by Moses the servant of God, and to observe and do all the commandments of the LORD our Lord, and his judgments and his statutes;"],
['Nehemiah','Neh','10','030','30',"And that we would not give our daughters unto the people of the land, nor take their daughters for our sons:"],
['Nehemiah','Neh','10','031','31',"And \emph{if} the people of the land bring ware or any victuals on the sabbath day to sell, \emph{that} we would not buy it of them on the sabbath, or on the holy day: and \emph{that} we would leave the seventh year, and the exaction of every debt."],
['Nehemiah','Neh','10','032','32',"Also we made ordinances for us, to charge ourselves yearly with the third part of a shekel for the service of the house of our God;"],
['Nehemiah','Neh','10','033','33',"For the shewbread, and for the continual meat offering, and for the continual burnt offering, of the sabbaths, of the new moons, for the set feasts, and for the holy \emph{things}, and for the sin offerings to make an atonement for Israel, and \emph{for} all the work of the house of our God."],
['Nehemiah','Neh','10','034','34',"And we cast the lots among the priests, the Levites, and the people, for the wood offering, to bring \emph{it} into the house of our God, after the houses of our fathers, at times appointed year by year, to burn upon the altar of the LORD our God, as \emph{it} \emph{is} written in the law:"],
['Nehemiah','Neh','10','035','35',"And to bring the firstfruits of our ground, and the firstfruits of all fruit of all trees, year by year, unto the house of the LORD:"],
['Nehemiah','Neh','10','036','36',"Also the firstborn of our sons, and of our cattle, as \emph{it} \emph{is} written in the law, and the firstlings of our herds and of our flocks, to bring to the house of our God, unto the priests that minister in the house of our God:"],
['Nehemiah','Neh','10','037','37',"And \emph{that} we should bring the firstfruits of our dough, and our offerings, and the fruit of all manner of trees, of wine and of oil, unto the priests, to the chambers of the house of our God; and the tithes of our ground unto the Levites, that the same Levites might have the tithes in all the cities of our tillage."],
['Nehemiah','Neh','10','038','38',"And the priest the son of Aaron shall be with the Levites, when the Levites take tithes: and the Levites shall bring up the tithe of the tithes unto the house of our God, to the chambers, into the treasure house."],
['Nehemiah','Neh','10','039','39',"For the children of Israel and the children of Levi shall bring the offering of the corn, of the new wine, and the oil, unto the chambers, where \emph{are} the vessels of the sanctuary, and the priests that minister, and the porters, and the singers: and we will not forsake the house of our God."]
]

Nehemiah11 = [
['Nehemiah','Neh','11','001','1',"And the rulers of the people dwelt at Jerusalem: the rest of the people also cast lots, to bring one of ten to dwell in Jerusalem the holy city, and nine parts \emph{to} \emph{dwell} in \emph{other} cities."],
['Nehemiah','Neh','11','002','2',"And the people blessed all the men, that willingly offered themselves to dwell at Jerusalem."],
['Nehemiah','Neh','11','003','3',"Now these \emph{are} the chief of the province that dwelt in Jerusalem: but in the cities of Judah dwelt every one in his possession in their cities, \emph{to} \emph{wit}, Israel, the priests, and the Levites, and the Nethinims, and the children of Solomon's servants."],
['Nehemiah','Neh','11','004','4',"And at Jerusalem dwelt \emph{certain} of the children of Judah, and of the children of Benjamin. Of the children of Judah; Athaiah the son of Uzziah, the son of Zechariah, the son of Amariah, the son of Shephatiah, the son of Mahalaleel, of the children of Perez;"],
['Nehemiah','Neh','11','005','5',"And Maaseiah the son of Baruch, the son of Col-hozeh, the son of Hazaiah, the son of Adaiah, the son of Joiarib, the son of Zechariah, the son of Shiloni."],
['Nehemiah','Neh','11','006','6',"All the sons of Perez that dwelt at Jerusalem \emph{were} four hundred threescore and eight valiant men."],
['Nehemiah','Neh','11','007','7',"And these \emph{are} the sons of Benjamin; Sallu the son of Meshullam, the son of Joed, the son of Pedaiah, the son of Kolaiah, the son of Maaseiah, the son of Ithiel, the son of Jesaiah."],
['Nehemiah','Neh','11','008','8',"And after him Gabbai, Sallai, nine hundred twenty and eight."],
['Nehemiah','Neh','11','009','9',"And Joel the son of Zichri \emph{was} their overseer: and Judah the son of Senuah \emph{was} second over the city."],
['Nehemiah','Neh','11','010','10',"Of the priests: Jedaiah the son of Joiarib, Jachin."],
['Nehemiah','Neh','11','011','11',"Seraiah the son of Hilkiah, the son of Meshullam, the son of Zadok, the son of Meraioth, the son of Ahitub, \emph{was} the ruler of the house of God."],
['Nehemiah','Neh','11','012','12',"And their brethren that did the work of the house \emph{were} eight hundred twenty and two: and Adaiah the son of Jeroham, the son of Pelaliah, the son of Amzi, the son of Zechariah, the son of Pashur, the son of Malchiah,"],
['Nehemiah','Neh','11','013','13',"And his brethren, chief of the fathers, two hundred forty and two: and Amashai the son of Azareel, the son of Ahasai, the son of Meshillemoth, the son of Immer,"],
['Nehemiah','Neh','11','014','14',"And their brethren, mighty men of valour, an hundred twenty and eight: and their overseer \emph{was} Zabdiel, the son of \emph{one} \emph{of} the great men."],
['Nehemiah','Neh','11','015','15',"Also of the Levites: Shemaiah the son of Hashub, the son of Azrikam, the son of Hashabiah, the son of Bunni;"],
['Nehemiah','Neh','11','016','16',"And Shabbethai and Jozabad, of the chief of the Levites, \emph{had} the oversight of the outward business of the house of God."],
['Nehemiah','Neh','11','017','17',"And Mattaniah the son of Micha, the son of Zabdi, the son of Asaph, \emph{was} the principal to begin the thanksgiving in prayer: and Bakbukiah the second among his brethren, and Abda the son of Shammua, the son of Galal, the son of Jeduthun."],
['Nehemiah','Neh','11','018','18',"All the Levites in the holy city \emph{were} two hundred fourscore and four."],
['Nehemiah','Neh','11','019','19',"Moreover the porters, Akkub, Talmon, and their brethren that kept the gates, \emph{were} an hundred seventy and two."],
['Nehemiah','Neh','11','020','20',"And the residue of Israel, of the priests, \emph{and} the Levites, \emph{were} in all the cities of Judah, every one in his inheritance."],
['Nehemiah','Neh','11','021','21',"But the Nethinims dwelt in Ophel: and Ziha and Gispa \emph{were} over the Nethinims."],
['Nehemiah','Neh','11','022','22',"The overseer also of the Levites at Jerusalem \emph{was} Uzzi the son of Bani, the son of Hashabiah, the son of Mattaniah, the son of Micha. Of the sons of Asaph, the singers \emph{were} over the business of the house of God."],
['Nehemiah','Neh','11','023','23',"For \emph{it} \emph{was} the king's commandment concerning them, that a certain portion should be for the singers, due for every day."],
['Nehemiah','Neh','11','024','24',"And Pethahiah the son of Meshezabeel, of the children of Zerah the son of Judah, \emph{was} at the king's hand in all matters concerning the people."],
['Nehemiah','Neh','11','025','25',"And for the villages, with their fields, \emph{some} of the children of Judah dwelt at Kirjath-arba, and \emph{in} the villages thereof, and at Dibon, and \emph{in} the villages thereof, and at Jekabzeel, and \emph{in} the villages thereof,"],
['Nehemiah','Neh','11','026','26',"And at Jeshua, and at Moladah, and at Beth-phelet,"],
['Nehemiah','Neh','11','027','27',"And at Hazar-shual, and at Beer-sheba, and \emph{in} the villages thereof,"],
['Nehemiah','Neh','11','028','28',"And at Ziklag, and at Mekonah, and in the villages thereof,"],
['Nehemiah','Neh','11','029','29',"And at En-rimmon, and at Zareah, and at Jarmuth,"],
['Nehemiah','Neh','11','030','30',"Zanoah, Adullam, and \emph{in} their villages, at Lachish, and the fields thereof, at Azekah, and \emph{in} the villages thereof. And they dwelt from Beer-sheba unto the valley of Hinnom."],
['Nehemiah','Neh','11','031','31',"The children also of Benjamin from Geba \emph{dwelt} at Michmash, and Aija, and Beth-el, and \emph{in} their villages,"],
['Nehemiah','Neh','11','032','32',"\emph{And} at Anathoth, Nob, Ananiah,"],
['Nehemiah','Neh','11','033','33',"Hazor, Ramah, Gittaim,"],
['Nehemiah','Neh','11','034','34',"Hadid, Zeboim, Neballat,"],
['Nehemiah','Neh','11','035','35',"Lod, and Ono, the valley of craftsmen."],
['Nehemiah','Neh','11','036','36',"And of the Levites \emph{were} divisions \emph{in} Judah, \emph{and} in Benjamin."]
]

Nehemiah12 = [
['Nehemiah','Neh','12','001','1',"Now these \emph{are} the priests and the Levites that went up with Zerubbabel the son of Shealtiel, and Jeshua: Seraiah, Jeremiah, Ezra,"],
['Nehemiah','Neh','12','002','2',"Amariah, Malluch, Hattush,"],
['Nehemiah','Neh','12','003','3',"Shechaniah, Rehum, Meremoth,"],
['Nehemiah','Neh','12','004','4',"Iddo, Ginnetho, Abijah,"],
['Nehemiah','Neh','12','005','5',"Miamin, Maadiah, Bilgah,"],
['Nehemiah','Neh','12','006','6',"Shemaiah, and Joiarib, Jedaiah,"],
['Nehemiah','Neh','12','007','7',"Sallu, Amok, Hilkiah, Jedaiah. These \emph{were} the chief of the priests and of their brethren in the days of Jeshua."],
['Nehemiah','Neh','12','008','8',"Moreover the Levites: Jeshua, Binnui, Kadmiel, Sherebiah, Judah, \emph{and} Mattaniah, \emph{which} \emph{was} over the thanksgiving, he and his brethren."],
['Nehemiah','Neh','12','009','9',"Also Bakbukiah and Unni, their brethren, \emph{were} over against them in the watches."],
['Nehemiah','Neh','12','010','10',"And Jeshua begat Joiakim, Joiakim also begat Eliashib, and Eliashib begat Joiada,"],
['Nehemiah','Neh','12','011','11',"And Joiada begat Jonathan, and Jonathan begat Jaddua."],
['Nehemiah','Neh','12','012','12',"And in the days of Joiakim were priests, the chief of the fathers: of Seraiah, Meraiah; of Jeremiah, Hananiah;"],
['Nehemiah','Neh','12','013','13',"Of Ezra, Meshullam; of Amariah, Jehohanan;"],
['Nehemiah','Neh','12','014','14',"Of Melicu, Jonathan; of Shebaniah, Joseph;"],
['Nehemiah','Neh','12','015','15',"Of Harim, Adna; of Meraioth, Helkai;"],
['Nehemiah','Neh','12','016','16',"Of Iddo, Zechariah; of Ginnethon, Meshullam;"],
['Nehemiah','Neh','12','017','17',"Of Abijah, Zichri; of Miniamin, of Moadiah, Piltai;"],
['Nehemiah','Neh','12','018','18',"Of Bilgah, Shammua; of Shemaiah, Jehonathan;"],
['Nehemiah','Neh','12','019','19',"And of Joiarib, Mattenai; of Jedaiah, Uzzi;"],
['Nehemiah','Neh','12','020','20',"Of Sallai, Kallai; of Amok, Eber;"],
['Nehemiah','Neh','12','021','21',"Of Hilkiah, Hashabiah; of Jedaiah, Nethaneel."],
['Nehemiah','Neh','12','022','22',"The Levites in the days of Eliashib, Joiada, and Johanan, and Jaddua, \emph{were} recorded chief of the fathers: also the priests, to the reign of Darius the Persian."],
['Nehemiah','Neh','12','023','23',"The sons of Levi, the chief of the fathers, \emph{were} written in the book of the chronicles, even until the days of Johanan the son of Eliashib."],
['Nehemiah','Neh','12','024','24',"And the chief of the Levites: Hashabiah, Sherebiah, and Jeshua the son of Kadmiel, with their brethren over against them, to praise \emph{and} to give thanks, according to the commandment of David the man of God, ward over against ward."],
['Nehemiah','Neh','12','025','25',"Mattaniah, and Bakbukiah, Obadiah, Meshullam, Talmon, Akkub, \emph{were} porters keeping the ward at the thresholds of the gates."],
['Nehemiah','Neh','12','026','26',"These \emph{were} in the days of Joiakim the son of Jeshua, the son of Jozadak, and in the days of Nehemiah the governor, and of Ezra the priest, the scribe."],
['Nehemiah','Neh','12','027','27',"And at the dedication of the wall of Jerusalem they sought the Levites out of all their places, to bring them to Jerusalem, to keep the dedication with gladness, both with thanksgivings, and with singing, \emph{with} cymbals, psalteries, and with harps."],
['Nehemiah','Neh','12','028','28',"And the sons of the singers gathered themselves together, both out of the plain country round about Jerusalem, and from the villages of Netophathi;"],
['Nehemiah','Neh','12','029','29',"Also from the house of Gilgal, and out of the fields of Geba and Azmaveth: for the singers had builded them villages round about Jerusalem."],
['Nehemiah','Neh','12','030','30',"And the priests and the Levites purified themselves, and purified the people, and the gates, and the wall."],
['Nehemiah','Neh','12','031','31',"Then I brought up the princes of Judah upon the wall, and appointed two great \emph{companies} \emph{of} \emph{them} \emph{that} \emph{gave} thanks, \emph{whereof} \emph{one} went on the right hand upon the wall toward the dung gate:"],
['Nehemiah','Neh','12','032','32',"And after them went Hoshaiah, and half of the princes of Judah,"],
['Nehemiah','Neh','12','033','33',"And Azariah, Ezra, and Meshullam,"],
['Nehemiah','Neh','12','034','34',"Judah, and Benjamin, and Shemaiah, and Jeremiah,"],
['Nehemiah','Neh','12','035','35',"And \emph{certain} of the priests' sons with trumpets; \emph{namely}, Zechariah the son of Jonathan, the son of Shemaiah, the son of Mattaniah, the son of Michaiah, the son of Zaccur, the son of Asaph:"],
['Nehemiah','Neh','12','036','36',"And his brethren, Shemaiah, and Azarael, Milalai, Gilalai, Maai, Nethaneel, and Judah, Hanani, with the musical instruments of David the man of God, and Ezra the scribe before them."],
['Nehemiah','Neh','12','037','37',"And at the fountain gate, which was over against them, they went up by the stairs of the city of David, at the going up of the wall, above the house of David, even unto the water gate eastward."],
['Nehemiah','Neh','12','038','38',"And the other \emph{company} \emph{of} \emph{them} \emph{that} \emph{gave} thanks went over against \emph{them}, and I after them, and the half of the people upon the wall, from beyond the tower of the furnaces even unto the broad wall;"],
['Nehemiah','Neh','12','039','39',"And from above the gate of Ephraim, and above the old gate, and above the fish gate, and the tower of Hananeel, and the tower of Meah, even unto the sheep gate: and they stood still in the prison gate."],
['Nehemiah','Neh','12','040','40',"So stood the two \emph{companies} \emph{of} \emph{them} \emph{that} \emph{gave} thanks in the house of God, and I, and the half of the rulers with me:"],
['Nehemiah','Neh','12','041','41',"And the priests; Eliakim, Maaseiah, Miniamin, Michaiah, Elioenai, Zechariah, \emph{and} Hananiah, with trumpets;"],
['Nehemiah','Neh','12','042','42',"And Maaseiah, and Shemaiah, and Eleazar, and Uzzi, and Jehohanan, and Malchijah, and Elam, and Ezer. And the singers sang loud, with Jezrahiah \emph{their} overseer."],
['Nehemiah','Neh','12','043','43',"Also that day they offered great sacrifices, and rejoiced: for God had made them rejoice with great joy: the wives also and the children rejoiced: so that the joy of Jerusalem was heard even afar off."],
['Nehemiah','Neh','12','044','44',"And at that time were some appointed over the chambers for the treasures, for the offerings, for the firstfruits, and for the tithes, to gather into them out of the fields of the cities the portions of the law for the priests and Levites: for Judah rejoiced for the priests and for the Levites that waited."],
['Nehemiah','Neh','12','045','45',"And both the singers and the porters kept the ward of their God, and the ward of the purification, according to the commandment of David, \emph{and} of Solomon his son."],
['Nehemiah','Neh','12','046','46',"For in the days of David and Asaph of old \emph{there} \emph{were} chief of the singers, and songs of praise and thanksgiving unto God."],
['Nehemiah','Neh','12','047','47',"And all Israel in the days of Zerubbabel, and in the days of Nehemiah, gave the portions of the singers and the porters, every day his portion: and they sanctified \emph{holy} \emph{things} unto the Levites; and the Levites sanctified \emph{them} unto the children of Aaron."]
]

Nehemiah13 = [
['Nehemiah','Neh','13','001','1',"On that day they read in the book of Moses in the audience of the people; and therein was found written, that the Ammonite and the Moabite should not come into the congregation of God for ever;"],
['Nehemiah','Neh','13','002','2',"Because they met not the children of Israel with bread and with water, but hired Balaam against them, that he should curse them: howbeit our God turned the curse into a blessing."],
['Nehemiah','Neh','13','003','3',"Now it came to pass, when they had heard the law, that they separated from Israel all the mixed multitude."],
['Nehemiah','Neh','13','004','4',"And before this, Eliashib the priest, having the oversight of the chamber of the house of our God, \emph{was} allied unto Tobiah:"],
['Nehemiah','Neh','13','005','5',"And he had prepared for him a great chamber, where aforetime they laid the meat offerings, the frankincense, and the vessels, and the tithes of the corn, the new wine, and the oil, which was commanded \emph{to} \emph{be} \emph{given} to the Levites, and the singers, and the porters; and the offerings of the priests."],
['Nehemiah','Neh','13','006','6',"But in all this \emph{time} was not I at Jerusalem: for in the two and thirtieth year of Artaxerxes king of Babylon came I unto the king, and after certain days obtained I leave of the king:"],
['Nehemiah','Neh','13','007','7',"And I came to Jerusalem, and understood of the evil that Eliashib did for Tobiah, in preparing him a chamber in the courts of the house of God."],
['Nehemiah','Neh','13','008','8',"And it grieved me sore: therefore I cast forth all the household stuff of Tobiah out of the chamber."],
['Nehemiah','Neh','13','009','9',"Then I commanded, and they cleansed the chambers: and thither brought I again the vessels of the house of God, with the meat offering and the frankincense."],
['Nehemiah','Neh','13','010','10',"And I perceived that the portions of the Levites had not been given \emph{them}: for the Levites and the singers, that did the work, were fled every one to his field."],
['Nehemiah','Neh','13','011','11',"Then contended I with the rulers, and said, Why is the house of God forsaken? And I gathered them together, and set them in their place."],
['Nehemiah','Neh','13','012','12',"Then brought all Judah the tithe of the corn and the new wine and the oil unto the treasuries."],
['Nehemiah','Neh','13','013','13',"And I made treasurers over the treasuries, Shelemiah the priest, and Zadok the scribe, and of the Levites, Pedaiah: and next to them \emph{was} Hanan the son of Zaccur, the son of Mattaniah: for they were counted faithful, and their office \emph{was} to distribute unto their brethren."],
['Nehemiah','Neh','13','014','14',"Remember me, O my God, concerning this, and wipe not out my good deeds that I have done for the house of my God, and for the offices thereof."],
['Nehemiah','Neh','13','015','15',"In those days saw I in Judah \emph{some} treading wine presses on the sabbath, and bringing in sheaves, and lading asses; as also wine, grapes, and figs, and all \emph{manner} \emph{of} burdens, which they brought into Jerusalem on the sabbath day: and I testified \emph{against} \emph{them} in the day wherein they sold victuals."],
['Nehemiah','Neh','13','016','16',"There dwelt men of Tyre also therein, which brought fish, and all manner of ware, and sold on the sabbath unto the children of Judah, and in Jerusalem."],
['Nehemiah','Neh','13','017','17',"Then I contended with the nobles of Judah, and said unto them, What evil thing \emph{is} this that ye do, and profane the sabbath day?"],
['Nehemiah','Neh','13','018','18',"Did not your fathers thus, and did not our God bring all this evil upon us, and upon this city? yet ye bring more wrath upon Israel by profaning the sabbath."],
['Nehemiah','Neh','13','019','19',"And it came to pass, that when the gates of Jerusalem began to be dark before the sabbath, I commanded that the gates should be shut, and charged that they should not be opened till after the sabbath: and \emph{some} of my servants set I at the gates, \emph{that} there should no burden be brought in on the sabbath day."],
['Nehemiah','Neh','13','020','20',"So the merchants and sellers of all kind of ware lodged without Jerusalem once or twice."],
['Nehemiah','Neh','13','021','21',"Then I testified against them, and said unto them, Why lodge ye about the wall? if ye do \emph{so} again, I will lay hands on you. From that time forth came they no \emph{more} on the sabbath."],
['Nehemiah','Neh','13','022','22',"And I commanded the Levites that they should cleanse themselves, and \emph{that} they should come \emph{and} keep the gates, to sanctify the sabbath day. Remember me, O my God, \emph{concerning} this also, and spare me according to the greatness of thy mercy."],
['Nehemiah','Neh','13','023','23',"In those days also saw I Jews \emph{that} had married wives of Ashdod, of Ammon, \emph{and} of Moab:"],
['Nehemiah','Neh','13','024','24',"And their children spake half in the speech of Ashdod, and could not speak in the Jews' language, but according to the language of each people."],
['Nehemiah','Neh','13','025','25',"And I contended with them, and cursed them, and smote certain of them, and plucked off their hair, and made them swear by God, \emph{saying}, Ye shall not give your daughters unto their sons, nor take their daughters unto your sons, or for yourselves."],
['Nehemiah','Neh','13','026','26',"Did not Solomon king of Israel sin by these things? yet among many nations was there no king like him, who was beloved of his God, and God made him king over all Israel: nevertheless even him did outlandish women cause to sin."],
['Nehemiah','Neh','13','027','27',"Shall we then hearken unto you to do all this great evil, to transgress against our God in marrying strange wives?"],
['Nehemiah','Neh','13','028','28',"And \emph{one} of the sons of Joiada, the son of Eliashib the high priest, \emph{was} son in law to Sanballat the Horonite: therefore I chased him from me."],
['Nehemiah','Neh','13','029','29',"Remember them, O my God, because they have defiled the priesthood, and the covenant of the priesthood, and of the Levites."],
['Nehemiah','Neh','13','030','30',"Thus cleansed I them from all strangers, and appointed the wards of the priests and the Levites, every one in his business;"],
['Nehemiah','Neh','13','031','31',"And for the wood offering, at times appointed, and for the firstfruits. Remember me, O my God, for good."]
]


def main(): 
    ProcessVerses('Nehemiah11',Nehemiah11)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And the rulers of the people dwelt at Jerusalem: the rest of the people also cast lots, to bring one of ten to dwell in Jerusalem the holy city, and nine parts \emph{to} \emph{dwell} in \emph{other} cities.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And the people blessed all the men, that willingly offered themselves to dwell at Jerusalem.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{Now these \emph{are} the chief of the province that dwelt in Jerusalem: but in the cities of Judah dwelt every one in his possession in their cities, \emph{to} \emph{wit}, Israel, the priests, and the Levites, and the Nethinims, and the children of Solomon's servants.}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And at Jerusalem dwelt \emph{certain} of the children of Judah, and of the children of Benjamin. Of the ch

Jeroham & 1 \\ \hline
Jerusalem & 7 \\ \hline
Jesaiah & 1 \\ \hline
Jeshua & 1 \\ \hline
Joed & 1 \\ \hline
Joel & 1 \\ \hline
Joiarib & 2 \\ \hline
Jozabad & 1 \\ \hline
Judah & 8 \\ \hline
Kirjath-arba & 1 \\ \hline
Kolaiah & 1 \\ \hline
Lachish & 1 \\ \hline
Levites & 7 \\ \hline
Lod & 1 \\ \hline
Maaseiah & 2 \\ \hline
Mahalaleel & 1 \\ \hline
Malchiah & 1 \\ \hline
Mattaniah & 2 \\ \hline
Mekonah & 1 \\ \hline
Meraioth & 1 \\ \hline
Meshezabeel & 1 \\ \hline
Meshillemoth & 1 \\ \hline
Meshullam & 2 \\ \hline
Micha & 2 \\ \hline
Michmash & 1 \\ \hline
Moladah & 1 \\ \hline
Moreover & 1 \\ \hline
Neballat & 1 \\ \hline
Nethinims & 3 \\ \hline
Nob & 1 \\ \hline
Now & 1 \\ \hline
Of & 3 \\ \hline
Ono & 1 \\ \hline
Ophel & 1 \\ \hline
Pashur & 1 \\ \hline
Pedaiah & 1 \\ \hline
Pelaliah & 1 \\ \hline
Perez & 2 \\ \hline
Pethahiah & 1 \\ \hline
Ramah & 1 \\ \hline
Sallai & 1 \\ \hline
Sallu & 1 \\ \hline
Senuah & 1 \\ \hline
Seraiah & 1 \\ \hline
Shabbethai & 1 \\ \hline
Shammua & 1 \\ \